In [1]:
import xgboost as xgb
import dolphindb 
import shioaji as sj

In [28]:
import json
with open("login.json", "r") as f:
    login_kwargs = json.loads(f.read())

In [29]:
api = sj.Shioaji()
accounts = api.login(**login_kwargs, contracts_cb=print)

Response Code: 0 | Event Code: 0 | Info: host '203.66.91.161:80', IP 203.66.91.161:80 (host 1 of 1) (host connection attempt 1 of 1) (total connection attempt 1 of 1) | Event: Session up
SecurityType.Index
SecurityType.Future
SecurityType.Option
SecurityType.Stock


In [2]:
ddb = dolphindb.session()
ddb.connect("localhost", 8828, "admin", "123456")

True

In [9]:
ddb.run("use ta")

In [6]:
LOAD_OHLCV_TABLE = """
ohlc_tick = loadTable("dfs://ohlc", `ohlc)
"""

In [7]:
ddb.run(LOAD_OHLCV_TABLE)

In [30]:
ddb.run("""
select top 5 * from ohlc_tick
""")

,Code,Date,Time,Open,High,Low,Close,volume
0,TXFG0,2020-06-24,1970-01-01 16:59:00,11475.0,11476.0,11474.0,11475.0,20
1,TXFG0,2020-06-24,1970-01-01 17:00:00,11476.0,11476.0,11475.0,11476.0,28
2,TXFG0,2020-06-24,1970-01-01 17:01:00,11476.0,11478.0,11475.0,11477.0,28
3,TXFG0,2020-06-24,1970-01-01 17:02:00,11477.0,11478.0,11477.0,11477.0,49
4,TXFG0,2020-06-24,1970-01-01 17:03:00,11478.0,11479.0,11477.0,11478.0,62


In [13]:
df = ddb.run("""
temptb = select Date, Time, Code, Open, High, Low, Close, Volume, 
       sma(Close, 20) as ma20, sma(Close, 60) as ma60,
       sma(Close, 20) - sma(Close, 60) as madiff,
       // bBands(Close, 20, 2, 2, 0) as `upband`ma`downband, 
       natr(High, Low, Close, 20) as NATR,
       (move(Close, -30) - Close) / Close as Label 
       from ohlc_tick context by Code
select *, move(madiff, 1) as madiff1 from temptb
""")

In [17]:
model = xgb.Booster()

In [18]:
model.load_model('model.xgb')

In [22]:
data = xgb.DMatrix(df.iloc[-1:][["madiff", "madiff1", "NATR"]])

In [23]:
if model.predict(data)[0] >= 0.03:
    api.place_order(api.Contracts.Futures[df['Code'].iloc[-1]],
                    sj.Order(
                        price=0, 
                        quantity=1, 
                        action=sj.constant.Action.Buy, 
                        price_type=sj.constant.FuturesPriceType.MKT,
                        order_type=sj.constant.FuturesOrderType.ROD,
                    )
                    )


array([-0.00229013], dtype=float32)